In [1]:
import torch as pt
from src.pupieapp_metric.pu_pieapp.models.common import PUPieAPP

weights = './src/resources/pupieapp_weights.pt'
if pt.cuda.is_available():
    map_location = 'cuda:0'
else:
    map_location = 'cpu'
state = pt.load(weights, map_location=map_location)
net = PUPieAPP(state)
net.eval()

PUPieAPP(
  (pu_transformer): PUTransformer()
  (dim_transformer): DimTransformer()
  (extractor): FeatureExtractor(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1,

In [5]:
import os

original_and_reconstructed_path = './original_and_reconstructed_images'
image_names = [image_name for image_name in os.listdir(original_and_reconstructed_path) if 'reconstructed' in image_name]

In [6]:
dynamic_range = 'sdr'
lum_top = 100
lum_bottom = 0.5
stride = 32
scores = {}

In [7]:
import imageio

for reconstructed_image_name in image_names:
    original_image_name = reconstructed_image_name.split('_')[0] + '.jpg'
    original_image_path = original_and_reconstructed_path + "/" + original_image_name
    original_image = imageio.v2.imread(original_image_path)
    original_image = pt.from_numpy(imageio.core.asarray(original_image))
    original_image = original_image.permute(2, 0, 1)
    original_image = original_image.unsqueeze(0)
    
    reconstructed_image_path = original_and_reconstructed_path + "/" + reconstructed_image_name
    reconstructed_image = imageio.v2.imread(reconstructed_image_path)
    reconstructed_image = pt.from_numpy(imageio.core.asarray(reconstructed_image))
    reconstructed_image = reconstructed_image.permute(2, 0, 1)
    reconstructed_image = reconstructed_image.unsqueeze(0)
    print(f"Original image name: {original_image_name}, reconstructed image name: {reconstructed_image_name}.")
    with pt.no_grad():
        scores[(original_image_name, reconstructed_image_name)] = net(original_image, 
                                                                      reconstructed_image, 
                                                                      im_type=dynamic_range, 
                                                                      lum_bottom=lum_bottom, 
                                                                      lum_top=lum_top, 
                                                                      stride=stride)

Original image name: image102.jpg, reconstructed image name: image102_reconstructed.jpg.


In [9]:
for key in scores.keys():
    print(f"Original_image: {key[0]}, reconstructed_image: {key[1]}, metric score: {scores[key].item()}.")

Original_image: image102.jpg, reconstructed_image: image102_reconstructed.jpg, metric score: -3.3574931621551514.
